# Optimize Portfolio Based on Forecast
We will use the forecasted data from Task 3 to optimize a sample investment portfolio consisting of three assets: 
- Tesla (TSLA)
- Vanguard Total Bond Market ETF (BND) 
- S&P 500 ETF (SPY). 

The goal is to adjust the portfolio weights to maximize returns while minimizing risks based on the predicted market trends.

In [1]:
import os, sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [7]:
from scripts.fetchPrepr_close_column import *
from scripts.portfolio_metrics import *
from scripts.optimize_analyze_portfolio import *
from scripts.visualize_portfolio import *
from scripts.lstm_model import *
from scripts.forecasting import *
from scripts.clean_data import *
from scripts.compute_and_forecast import *

In [9]:
# Fetch historical data
tickers = ['TSLA', 'BND', 'SPY']
start_date = '2015-01-01'
end_date = '2025-01-31'
data = fetch_data(tickers, start_date, end_date)

[*********************100%***********************]  3 of 3 completed


In [15]:
data['Close']

Ticker,BND,SPY,TSLA
Date,,,
2015-01-02,62.387089,172.592865,14.620667
2015-01-05,62.568237,169.475922,14.006000
2015-01-06,62.749397,167.879639,14.085333
2015-01-07,62.787163,169.971588,14.063333
2015-01-08,62.689007,172.987701,14.041333
...,...,...,...
2025-01-24,71.592484,607.969971,406.579987
2025-01-27,71.970116,599.369995,397.149994
2025-01-28,71.930367,604.520020,398.089996


In [5]:
tsla_data = fetch_data('TSLA', '2015-01-01', '2025-01-31')
train_data, test_data = preprocess_data(tsla_data)

# Load the trained LSTM model
lstm_model = load_lstm_model('lstm_model.keras')

# Load the scaler
scaler = joblib.load('scaler.pkl')

# Prepare the last sequence for forecasting
lookback = 60
last_sequence = train_data[-lookback:].values.reshape(-1, 1)

# Generate forecasts
forecast_valuesL = lstm_forecast(lstm_model, last_sequence, steps=12, lookback=lookback, scaler_filename='scaler.pkl')

# Use forecast_valuesL as the TSLA forecast
tsla_forecast = forecast_valuesL

[*********************100%***********************]  1 of 1 completed


LSTM model loaded from lstm_model.keras


In [16]:
# Create combined DataFrame
combined_data = create_combined_dataframe(tsla_forecast, data['Close']['BND'], data['Close']['SPY'])

In [17]:
# Compute annual returns and covariance matrix
annual_returns, covariance_matrix = compute_annual_returns_and_covariance(combined_data)